In [1]:
import os

# Subir un nivel respecto a la carpeta actual
os.chdir("..")

In [2]:
from Utils import load_parameters_file, from_matrix_to_preset, MSE, denormalize_preset, normalize_preset, pretty_print, PARAM_NAMES, from_preset_to_matrix, mel_spectrogram, mfcc, spectrogram
from Synth import Synth
import numpy as np
import cma
import matplotlib.pyplot as plt
from multiprocessing import Pool
from parallelEvaluation import evaluate_presets
from globals import SAMPLE_RATE, DURATION, PROCESSORS
from IPython.display import Audio

In [3]:
def render_presets(presets):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=presets
    )
    return synth.process_audio()

def evaluate_target(preset):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=preset
    )
    audio = synth.process_audio().astype(np.float64)
    # return mfcc(audio, sr=SAMPLE_RATE)
    return (0,mel_spectrogram(audio, sr=SAMPLE_RATE), audio )

In [4]:
target_audio = render_presets(load_parameters_file('target.json'))
Audio(target_audio[0], rate=SAMPLE_RATE)

In [ ]:
target_C = evaluate_target(load_parameters_file('target.json'))

def get_fitness(solutions, pool):
    solutions_splitted = np.split(solutions, PROCESSORS)
    presets_splitted = [(denormalize_preset(from_matrix_to_preset(chunk)), target_C) for chunk in solutions_splitted]
    
    solutions_evaluated = pool.map(evaluate_presets, presets_splitted)
    return np.concatenate(solutions_evaluated)

if __name__ == '__main__':
    optimizable_params = PARAM_NAMES

    bestever = cma.optimization_tools.BestSolution()

    x0 = np.random.rand(len(optimizable_params))
    sigma0 = 0.5

    with Pool(PROCESSORS) as pool:
        for lam in [15, 15, 30, 30, 64]:
            options = {
                'popsize': lam,
                'verb_append': bestever.evalsall,
                'bounds': [np.zeros_like(x0), np.ones_like(x0)],
                'tolfun': 1e-8
            }

            es = cma.CMAEvolutionStrategy(x0, sigma0, options)
            gen = 1

            while not es.stop():
                solutions = np.array(es.ask())  # devuelve una lista de individuos

                fitnesses = get_fitness(solutions, pool)

                best_idx = np.argmin(fitnesses)          # índice del mejor fitness
                best_solution = solutions[best_idx]      # solución correspondiente
                best_fitness = fitnesses[best_idx]       # fitness correspondiente

                print("Gen", gen, "Mejor fitness:", best_fitness, "Sigma", es.sigma)

                es.tell(solutions, fitnesses)  # pasar fitness al algoritmo

                gen += 1

                # if gen % 2000 == 0:
                #     r = input('Terminar?')
                #     if r == 'yes':
                #         break

            bestever.update(es.best)

            print('termination:', es.stop())
            cma.s.pprint(es.best.__dict__)

            bestever.update(es.best)

            if bestever.f < 1e-5:  # si se alcanza el óptimo global
                break

            x0 = bestever.get()[0]
            sigma0 = bestever.sigma * 10

    # Mejor solución
    best_solution = bestever.get()[0]
    print("Mejor individuo:", best_solution)

c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\evolution_strategy.py:1531: UserWarning: Sampling standard deviation i=0 (and 41 others) at iteration 0 multiplied by 0.6666666666666666 to stds[0]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={0}{4} at iteration {1}"


(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 42 (seed=680077, Thu Jan 15 19:37:40 2026)
Gen 1 Mejor fitness: 0.07179365187968667 Sigma 0.5
Gen 2 Mejor fitness: 0.0433625874280304 Sigma 0.475436526390064
Gen 3 Mejor fitness: 0.05710659554737557 Sigma 0.46655304314390045
Gen 4 Mejor fitness: 0.04497822751467324 Sigma 0.4602414006673243
Gen 5 Mejor fitness: 0.03915043497823714 Sigma 0.4537019190535553
Gen 6 Mejor fitness: 0.04582993365268342 Sigma 0.4500018823265947
Gen 7 Mejor fitness: 0.055637625772533894 Sigma 0.44314359270094045
Gen 8 Mejor fitness: 0.046860604473356104 Sigma 0.4348664649602434
Gen 9 Mejor fitness: 0.054678416070870206 Sigma 0.43077067196674795
Gen 10 Mejor fitness: 0.048272396025162374 Sigma 0.42897655898731546
Gen 11 Mejor fitness: 0.05737521260472099 Sigma 0.42683764152326703
Gen 12 Mejor fitness: 0.052695560456728005 Sigma 0.42513289213625216
Gen 13 Mejor fitness: 0.0477663368620726 Sigma 0.42922687336935844
Gen 14 Mejor fitness: 0.05158887566718249 Sigma 0.42

KeyboardInterrupt: 

In [ ]:
# from deap import base, creator, tools, algorithms
# import array
# import random

# IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY = len(PARAM_NAMES), 0, 1, 0, 1

# def checkStrategy(minstrategy):
#     def decorator(func):
#         def wrappper(*args, **kargs):
#             children = func(*args, **kargs)
#             for child in children:
#                 for i, s in enumerate(child.strategy):
#                     if s < minstrategy:
#                         child.strategy[i] = minstrategy
#             return children
#         return wrappper
#     return decorator

# def checkBounds(minval, maxval):
#     def decorator(func):
#         def wrapper(*args, **kwargs):
#             children = func(*args, **kwargs)
#             for child in children:
#                 for i in range(len(child)):
#                     if child[i] < minval:
#                         child[i] = minval
#                     elif child[i] > maxval:
#                         child[i] = maxval
#             return children
#         return wrapper
#     return decorator

# creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
# creator.create("Strategy", array.array, typecode="d")

# def generateES(icls, scls, size, imin, imax, smin, smax):
#     ind = icls(random.uniform(imin, imax) for _ in range(size))
#     ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
#     return ind

# toolbox = base.Toolbox()

# toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
#     IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# toolbox.register("evaluate", evaluate_single)

# toolbox.register("mate", tools.cxESBlend, alpha=0.1)
# toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
# toolbox.register("select", tools.selTournament, tournsize=3)

# toolbox.decorate("mate", checkStrategy(MIN_STRATEGY))
# toolbox.decorate("mutate", checkStrategy(MIN_STRATEGY))

# toolbox.decorate("mate", checkBounds(MIN_VALUE, MAX_VALUE))
# toolbox.decorate("mutate", checkBounds(MIN_VALUE, MAX_VALUE))

# def main():
#     MU, LAMBDA = 10, 100
#     pop = toolbox.population(n=MU)
#     hof = tools.HallOfFame(1)
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     # stats.register("avg", np.mean)
#     # stats.register("std", np.std)
#     # stats.register("min", np.min)
#     stats.register("max", np.max)

#     pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
#         cxpb=0.6, mutpb=0.3, ngen=500, stats=stats, halloffame=hof, verbose=True)

#     # return pop, logbook, hof
#     print('El mejor individuo es', hof[0], evaluate_single(hof[0]))

# main()

In [ ]:
# solution = init_solution_mask
# for i, param in enumerate(optimizable_params):
#     idx = PARAM_NAMES.index(param)
#     solution[idx] = best_solution[i]

# Error [0.03556078]
# Error [0.00035253]



solution = np.expand_dims(best_solution, axis=0)
error = evaluate_presets((denormalize_preset(from_matrix_to_preset(solution)), target_C))
print('Error', error)

target_audio = render_presets(load_parameters_file('target.json'))
predicted_audio = render_presets(denormalize_preset(from_matrix_to_preset(solution)))

Error [0.00035253]


In [ ]:
from scipy.io import wavfile

NAME = "exports/s3_a8"

audio_int16 = np.int16(target_audio[0] / np.max(np.abs(target_audio[0])) * 32767)
wavfile.write(f"{NAME}_target.wav", SAMPLE_RATE, audio_int16)

Audio(target_audio[0], rate=SAMPLE_RATE)

In [ ]:
audio_int16 = np.int16(predicted_audio[0] / np.max(np.abs(predicted_audio[0])) * 32767)
wavfile.write(f"{NAME}_predicted_{error}.wav", SAMPLE_RATE, audio_int16)

Audio(predicted_audio[0], rate=SAMPLE_RATE)

In [ ]:
denormalized_predicted_preset = denormalize_preset(from_matrix_to_preset(solution))
pretty_print(denormalized_predicted_preset)

{
  lfo1_rate:   [
    0.017808691461902313
  ]
  lfo1_shape:   [
    1.9900419395587496
  ]
  lfo2_rate:   [
    0.05285433630268959
  ]
  lfo2_shape:   [
    1.8981274157757442
  ]
  osc1_shape:   [
    0.0019427585951958955
  ]
  osc1_phase:   [
    0.7088870035681971
  ]
  osc1_volume:   [
    4.684765243171842e-05
  ]
  osc1_freq:   [
    2760.7690579980126
  ]
  osc1_vdepth:   [
    0.3729167247584033
  ]
  osc1_pdepth:   [
    0.8242683683306985
  ]
  osc2_shape:   [
    2.151103496458367
  ]
  osc2_phase:   [
    0.08927635140732193
  ]
  osc2_volume:   [
    5.376971609640527e-06
  ]
  osc2_freq:   [
    3826.760470756387
  ]
  osc2_vdepth:   [
    0.13297053553629912
  ]
  osc2_pdepth:   [
    0.3433473392030614
  ]
  osc3_shape:   [
    0.7924962444416754
  ]
  osc3_phase:   [
    0.9233967322798057
  ]
  osc3_volume:   [
    7.306091438430396e-05
  ]
  osc3_freq:   [
    10668.56989429701
  ]
  osc3_vdepth:   [
    0.7909295916599572
  ]
  osc3_pdepth:   [
    0.41755885708

In [ ]:
np.diagonal(es.C)

array([8.41523085e+00, 2.30937624e+00, 6.66030308e-01, 9.60406256e+00,
       2.62566911e+00, 2.47785844e+00, 4.15211899e+00, 8.49970182e-01,
       9.41354771e-01, 1.16205508e+01, 1.64652091e+00, 2.02128685e+00,
       5.63551987e+00, 1.60335564e+00, 9.25308858e+00, 6.20238159e-04,
       1.06450443e+01, 8.89449375e+00, 7.14958221e-04, 2.33644679e-02,
       4.76413305e+00, 3.50017364e-07, 1.70037737e+00, 1.76712428e-01,
       1.04956034e+00, 8.44233777e+00, 2.56052296e+00, 2.18954549e+00,
       2.89903602e+00, 2.24662178e+00, 1.12610315e+00, 6.71780999e-04,
       5.80566617e+00, 4.09234857e+00, 5.36321180e+00, 2.64745188e-03,
       1.87452690e+00, 4.22234986e-03, 1.31221490e+00, 2.56922787e+00,
       7.48242999e+00, 3.10160463e+00])